In [1]:
import numpy as np
import pandas as pd
from small_text import TransformersDataset, TransformerModelArguments, LeastConfidence, TransformerBasedClassificationFactory as TransformerFactory, PoolBasedActiveLearner, random_initialization_balanced as init
from transformers import AutoTokenizer
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
from synergy_dataset import Dataset, iter_datasets

from imblearn.over_sampling import RandomOverSampler, SMOTEN
import torch

from sklearn.model_selection import train_test_split

from transformers import AutoTokenizer

import matplotlib.pyplot as plt

/home/junior/Mestrado Comp Ifes/Dissertação/small-text/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-27 07:32:16.825031: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-27 07:32:17.155050: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-27 07:32:18.033663: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/junior/Mestrado Comp Ifes/Dissertação/small-text/.venv/lib/python3.8/site-packages/pyalex/api.py:493: DeprecationWarning: Concep

In [2]:
dataset = Dataset('Radjenovic_2013')
dataset = dataset.to_frame()
# dataset = pd.read_csv('../datasets/synergy_dataset/Radjenovic_2013.csv')
dataset = dataset.dropna()

In [3]:
X = np.array(dataset['abstract'])
y = np.array(dataset['label_included'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [4]:
sampler = SMOTEN(random_state=42)
X_train_os, y_train_os = sampler.fit_resample(X_train.reshape(-1, 1), y_train)

In [5]:
# ros = RandomOverSampler(random_state=42)
# X_train_os, y_train_os = ros.fit_resample(X_train.reshape(-1, 1), y_train)
# X_train_os = np.concatenate(X_train_os, axis=0)

In [6]:
# transformer_model = 'allenai/specter2_base'
transformer_model = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(transformer_model)

In [7]:
train = TransformersDataset.from_arrays(X_train_os.flatten(), y_train_os, tokenizer, target_labels=np.array([0, 1]), max_length=256)
test = TransformersDataset.from_arrays(X_test, y_test, tokenizer, target_labels=np.array([0, 1]), max_length=256)
#max_length > 256 geralmente ultrapassa a RAM da GPU (8GB)

/home/junior/Mestrado Comp Ifes/Dissertação/small-text/.venv/lib/python3.8/site-packages/small_text/utils/annotations.py:67: ExperimentalWarning: The function from_arrays is experimental and maybe subject to change soon.
  warnings.warn(f'The {subject} {func_or_class.__name__} is experimental '


In [8]:
print('Treino:')
print(0, len(train.y[train.y==0])/len(train.y))
print(1, 1-len(train.y[train.y==0])/len(train.y))
print(f'Número de amostras sem oversampling: {len(X_train)}')
print(f'Número de amostras com oversampling: {len(train)}')

print('\nTeste:')
print(0, len(test.y[test.y==0])/len(test.y))
print(1, 1-len(test.y[test.y==0])/len(test.y))
print(f'Número de amostras: {len(X_test)}')

Treino:
0 0.5
1 0.5
Número de amostras sem oversampling: 3934
Número de amostras com oversampling: 7812

Teste:
0 0.9896800825593395
1 0.01031991744066052
Número de amostras: 1938


In [9]:
num_classes = 2
model_args = TransformerModelArguments(transformer_model)
clf_factory = TransformerFactory(model_args, num_classes, kwargs={'device': 'cuda'})
# clf_factory = TransformerFactory(model_args, num_classes)
query_strategy = LeastConfidence()

In [10]:
active_learner = PoolBasedActiveLearner(clf_factory, query_strategy, train)
indices_initial = init(train.y, n_samples=10)
active_learner.initialize_data(indices_initial, train.y[indices_initial])

In [11]:
num_queries = 10
results = []
indices_labeled = []

for i in range(num_queries):
    indices_queried = active_learner.query(num_samples=20)
    y = train.y[indices_queried]
    active_learner.update(y)

    indices_labeled = np.concatenate([indices_queried, indices_labeled])

    y_pred_train = active_learner.classifier.predict(train)
    y_pred_test = active_learner.classifier.predict(test)
    print(f'\nIteration {i+1} ({len(indices_labeled)} samples)')
    print('Train accuracy: {:.2f}'.format(accuracy_score(train.y, y_pred_train)))
    print('Train precision: {:.2f}'.format(precision_score(train.y, y_pred_train, zero_division=np.nan)))
    print('Train recall: {:.2f}'.format(recall_score(train.y, y_pred_train, zero_division=np.nan)))
    print('Train F1 score: {:.2f}'.format(f1_score(train.y, y_pred_train)))
    print('\n')
    print('Test accuracy: {:.2f}'.format(accuracy_score(test.y, y_pred_test)))
    print('Test precision: {:.2f}'.format(precision_score(test.y, y_pred_test, zero_division=np.nan)))
    print('Test recall: {:.2f}'.format(recall_score(test.y, y_pred_test, zero_division=np.nan)))
    print('Test F1 score: {:.2f}'.format(f1_score(test.y, y_pred_test)))

    results.append((y_pred_train, y_pred_test))


Iteration 1 (20 samples)
Train accuracy: 1.00
Train precision: 1.00
Train recall: 0.99
Train F1 score: 1.00


Test accuracy: 0.99
Test precision: 0.00
Test recall: 0.00
Test F1 score: 0.00

Iteration 2 (40 samples)
Train accuracy: 1.00
Train precision: 1.00
Train recall: 0.99
Train F1 score: 1.00


Test accuracy: 0.99
Test precision: nan
Test recall: 0.00
Test F1 score: 0.00


KeyboardInterrupt: 

In [ ]:
# transformer_model = 'allenai/specter2_base'
# # transformer_model = 'bert-base-uncased'
# tokenizer = AutoTokenizer.from_pretrained(transformer_model)

# train = TransformersDataset.from_arrays(X_train_os, y_train_os, tokenizer, target_labels=np.array([0, 1]), max_length=256)
# test = TransformersDataset.from_arrays(X_test, y_test, tokenizer, target_labels=np.array([0, 1]), max_length=256)

# num_classes = 2
# model_args = TransformerModelArguments(transformer_model)
# clf_factory = TransformerFactory(model_args, num_classes, kwargs={'device': 'cuda'})
# # clf_factory = TransformerFactory(model_args, num_classes)
# query_strategy = LeastConfidence()

# active_learner = PoolBasedActiveLearner(clf_factory, query_strategy, train)

# indices_initial = init(train.y, n_samples=4)

# active_learner.initialize_data(indices_initial, train.y[indices_initial])

# num_queries = 10

# for i in range(num_queries):
#     indices_queried = active_learner.query(num_samples=10)
#     y = train.y[indices_queried]
#     active_learner.update(y)

#     y_pred_train = active_learner.classifier.predict(train)
#     y_pred_test = active_learner.classifier.predict(test)
#     print(f'\nIteration {i+1}')
#     print('Train accuracy: {:.2f}'.format(accuracy_score(train.y, y_pred_train)))
#     print('Train precision: {:.2f}'.format(precision_score(train.y, y_pred_train, zero_division=np.nan)))
#     print('Train recall: {:.2f}'.format(recall_score(train.y, y_pred_train, zero_division=np.nan)))
#     print('Train F1 score: {:.2f}'.format(f1_score(train.y, y_pred_train)))
#     print('\n')
#     print('Test accuracy: {:.2f}'.format(accuracy_score(test.y, y_pred_test)))
#     print('Test precision: {:.2f}'.format(precision_score(test.y, y_pred_test, zero_division=np.nan)))
#     print('Test recall: {:.2f}'.format(recall_score(test.y, y_pred_test, zero_division=np.nan)))
#     print('Test F1 score: {:.2f}'.format(f1_score(test.y, y_pred_test)))

/home/junior/.local/lib/python3.10/site-packages/small_text/utils/annotations.py:67: ExperimentalWarning: The function from_arrays is experimental and maybe subject to change soon.
  warnings.warn(f'The {subject} {func_or_class.__name__} is experimental '



Iteration 1
Train accuracy: 0.50
Train precision: nan
Train recall: 0.00
Train F1 score: 0.00


Test accuracy: 0.99
Test precision: nan
Test recall: 0.00
Test F1 score: 0.00

Iteration 2
Train accuracy: 0.61
Train precision: 0.97
Train recall: 0.22
Train F1 score: 0.36


Test accuracy: 0.99
Test precision: 0.26
Test recall: 0.25
Test F1 score: 0.26

Iteration 3
Train accuracy: 0.62
Train precision: 0.98
Train recall: 0.25
Train F1 score: 0.40


Test accuracy: 0.98
Test precision: 0.19
Test recall: 0.15
Test F1 score: 0.17

Iteration 4
Train accuracy: 0.80
Train precision: 0.94
Train recall: 0.64
Train F1 score: 0.76


Test accuracy: 0.94
Test precision: 0.09
Test recall: 0.50
Test F1 score: 0.15

Iteration 5
Train accuracy: 0.61
Train precision: 1.00
Train recall: 0.22
Train F1 score: 0.36


Test accuracy: 0.99
Test precision: 0.50
Test recall: 0.05
Test F1 score: 0.09

Iteration 6
Train accuracy: 0.61
Train precision: 1.00
Train recall: 0.21
Train F1 score: 0.35


Test accuracy: 0.99